In [1]:
# al tope de tu notebook, _antes_ de cualquier otra importación
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from gluonts.dataset.common import ListDataset
from gluonts.torch.model.deepar import DeepAREstimator
from pytorch_lightning import Trainer
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from ray import tune
from ray.tune.schedulers import ASHAScheduler
import torch
from gluonts.torch.distributions import StudentTOutput
from ray.air import session
from functools import partial


c:\Users\m_sam\anaconda3\envs\ts-venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-03 03:59:41,549	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-07-03 03:59:41,696	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# 1) load
sell_in = pd.read_csv('../data/sell-in.txt', sep='\t', parse_dates=['periodo'])
prod_vigentes = pd.read_csv('../data/product_id_apredecir201912.txt', sep='\t')

C:\Users\m_sam\AppData\Local\Temp\ipykernel_50596\281954699.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sell_in = pd.read_csv('../data/sell-in.txt', sep='\t', parse_dates=['periodo'])


In [3]:
# 2) Convierte 'periodo' a datetime (si viene en formato YYYYMM)
sell_in['periodo'] = pd.to_datetime(
    sell_in['periodo'],
    format='%Y%m'
)

In [4]:
# 2) Agrega tn a nivel (periodo, product_id)
sell_in_agg = (
    sell_in
    .groupby(['periodo', 'product_id'], as_index=False)['tn']
    .sum()
)

In [5]:
sell_in_agg['periodo'] = (
    pd.to_datetime(sell_in_agg['periodo'], format="%Y-%m-%d")
)

In [6]:
# 2) constants
FREQ = "M"
PREDICTION_LENGTH = 2
NUM_SAMPLES = 100

In [7]:
# 3) Construye dataset + lista de pids
dataset = []
pids = []
for pid in prod_vigentes['product_id'].unique():
    # a) filtrar
    df = sell_in_agg.loc[
        sell_in_agg['product_id'] == pid,
        ['periodo', 'tn']
    ].set_index('periodo').sort_index()

    if df.empty:
        continue

    # b) remuestreo mensual sumando todas las ventas de cada mes
    monthly = df['tn'].resample(FREQ).sum()

    # c) rellenar meses sin datos con 0
    target  = monthly.fillna(0).values

    # d) armar la entrada del ListDataset
    dataset.append({
        "start": monthly.index[0],          # Timestamp('2017-01-31 00:00:00')
        "target": target.astype('float32'),
        "item_id": str(pid),
    })
    pids.append(pid)


train_ds = ListDataset(dataset, freq=FREQ)

In [ ]:
# def train_deepar(config):
#     # 1) Construye el estimador con los hiperparámetros que quieres
#     estimator = DeepAREstimator(
#         freq="M",
#         prediction_length=1,
#         # input_size=1,
#         hidden_size=config["hidden_size"],
#         num_layers=config["num_layers"],
#         lr=config["lr"],    
#         num_batches_per_epoch=config["epochs"],
#         batch_size=32,  # o también tunéalo si quieres
#         distr_output=StudentTOutput(),  # o la distribución que uses
#         scaling=True,
#     )
#     predictor = estimator.train(train_ds)
#     test_ds = ListDataset(
#         [
#             {
#                 "start": item["start"],
#                 # dejamos sólo la parte 'histórica' y eliminamos las últimas `PREDICTION_LENGTH` observaciones
#                 "target": item["target"][:-PREDICTION_LENGTH],
#                 "item_id": item["item_id"],
#             }
#             for item in dataset  # dataset debe ser la lista de diccionarios que usaste para train_ds
#         ],
#         freq=FREQ,
#     )
#     forecast_it, ts_it = make_evaluation_predictions(
#         dataset=test_ds,
#         predictor=predictor,
#         num_samples=100,
#     )
#     evaluator   = Evaluator(quantiles=[0.5])
#     agg_metrics, _ = evaluator(ts_it, forecast_it)
#     forecasts = list(forecast_it)
#     tss       = list(ts_it)
    
#     # 3) compute MAE for each series, then average
#     maes = []
#     for ts, fcst in zip(tss, forecasts):
#         y_true = ts[-PREDICTION_LENGTH:]
#         # fcst.samples.shape == (num_samples, prediction_length)
#         y_pred = np.mean(fcst.samples, axis=0)
#         maes.append(mean_absolute_error(y_true, y_pred))
    
#     mean_mae = float(np.mean(maes))
    
#     # 4) report back to Tune
#     # tune.track.log(MAE=mean_mae)
#     # tune.report(MAE=mean_mae)
#     session.report({"MAE": mean_mae})
    

# # define espacio de búsqueda
# search_space = {
#     "lr":        tune.loguniform(1e-4, 1e-2),
#     "epochs":    tune.choice([10, 20, 30]),
#     "num_layers":tune.choice([1, 2, 3]),
#     "hidden_size":tune.choice([20, 40, 80]),
# }

# scheduler = ASHAScheduler(
#     # metric="MAE",
#     # mode="min",
#     max_t=30,
#     grace_period=5,
# )

# # 2) indícale a Ray un directorio base MUY corto
# local_dir = r"C:\tmp\ray"
# os.makedirs(local_dir, exist_ok=True)

# # 3) crea nombres de sub-carpeta super-cortos para cada trial
# def short_dirname_creator(trial):
#     return f"trial_{trial.trial_id}"

# # 4) lanza el hyper-param search usando esas dos opciones
# analysis = tune.run(
#     train_deepar,
#     config=search_space,
#     num_samples=16,
#     scheduler=scheduler,
#     resources_per_trial={"cpu": 2},
#     local_dir=local_dir,                        # aquí
#     trial_dirname_creator=short_dirname_creator, # y aquí
#     metric="MAE",      # ← the name you used in tune.report(...)
#     mode="min",        # ← "min" for MAE, "max" for e.g. accuracy
# )

# print("Mejor configuración:", analysis.best_config)


2025-07-03 03:45:58,806	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


Trial name,MAE
train_deepar_60923_00000,nan
train_deepar_60923_00001,nan
train_deepar_60923_00002,nan
train_deepar_60923_00003,nan
train_deepar_60923_00004,nan
train_deepar_60923_00005,nan
train_deepar_60923_00006,nan
train_deepar_60923_00007,nan
train_deepar_60923_00008,nan
train_deepar_60923_00009,nan


2025-07-03 03:48:46,996	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to 'C:/Users/m_sam/ray_results/train_deepar_2025-07-03_03-45-58' in 0.0361s.
2025-07-03 03:48:47,034	INFO tune.py:1048 -- Total run time: 168.23 seconds (167.95 seconds for the tuning loop).
2025-07-03 03:48:47,054	WARNING experiment_analysis.py:568 -- Could not find best trial. Did you pass the correct `metric` parameter?


Mejor configuración: None


In [ ]:
def train_deepar(config, train_ds, raw_dataset):
    # 1) construye el estimador usando los hyperparámetros del config
    estimator = DeepAREstimator(
        freq=FREQ,
        prediction_length=PREDICTION_LENGTH,
        distr_output=StudentTOutput(),
        scaling=True,
        # aquí mapeamos tu "num_layers" y "hidden_size" (en GluonTS se llama num_cells)
        num_layers=config["num_layers"],
        hidden_size=config["hidden_size"],
        trainer_kwargs={
            "max_epochs": config["epochs"],
            # "accelerator": "gpu",
            # "devices": 1,
        },
    )

    # 2) entrena
    predictor = estimator.train(train_ds)

    # 3) prepara el test_ds truncando las últimas PREDICTION_LENGTH observaciones
    test_ds = ListDataset(
        [
            {
                "start": item["start"],
                "target": item["target"][:-PREDICTION_LENGTH],
                "item_id": item["item_id"],
            }
            for item in raw_dataset
        ],
        freq=FREQ,
    )

    # 4) genera predicciones
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=test_ds,
        predictor=predictor,
        num_samples=100,
    )

    forecasts = list(forecast_it)
    tss       = list(ts_it)

    # 5) calcula MAE promedio
    maes = []
    for ts, fcst in zip(tss, forecasts):
        y_true = ts[-PREDICTION_LENGTH:]
        y_pred = np.mean(fcst.samples, axis=0)
        maes.append(mean_absolute_error(y_true, y_pred))
    mean_mae = float(np.mean(maes))

    # 6) reporta el resultado a Ray Tune
    session.report({"MAE": mean_mae})


# ——— scheduler y espacio de búsqueda ———
scheduler = ASHAScheduler(
    max_t=30,
    grace_period=5,
)

search_space = {
    "lr":          tune.loguniform(1e-4, 1e-2),
    "epochs":      tune.choice([10, 20, 30]),
    "num_layers":  tune.choice([1, 2, 3]),
    "hidden_size": tune.choice([20, 40, 80]),
}

# ——— directorio corto para logs de Ray ———
local_dir = r"C:\tmp\ray"
os.makedirs(local_dir, exist_ok=True)

def short_dirname_creator(trial):
    return f"t{trial.trial_id}"


# ——— envolvemos nuestra función para inyectar los datasets ———
trainable = tune.with_parameters(
    train_deepar,
    train_ds=train_ds,
    raw_dataset=dataset,
)

# ——— lanzamos la búsqueda ———
analysis = tune.run(
    trainable,                          # <- NOTA: pasamos aquí el objeto retornado por with_parameters
    config=search_space,
    num_samples=16,
    scheduler=scheduler,
    resources_per_trial={"cpu": 2},
    local_dir=local_dir,
    trial_dirname_creator=short_dirname_creator,
    metric="MAE",   # nombre del campo que reportamos
    mode="min",     # queremos minimizar MAE
)

print("Mejor configuración encontrada:", analysis.best_config)

2025-07-03 04:24:17,636	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


Trial name,MAE
train_deepar_bac78_00000,11.5246
train_deepar_bac78_00001,11.3448
train_deepar_bac78_00002,10.5293
train_deepar_bac78_00003,14.0623
train_deepar_bac78_00004,10.135
train_deepar_bac78_00005,12.7884
train_deepar_bac78_00006,10.7748
train_deepar_bac78_00007,9.89062
train_deepar_bac78_00008,11.4331
train_deepar_bac78_00009,9.2675


2025-07-03 04:25:43,957	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to 'C:/Users/m_sam/ray_results/train_deepar_2025-07-03_04-24-17' in 0.0321s.
2025-07-03 04:25:43,995	INFO tune.py:1048 -- Total run time: 86.36 seconds (86.27 seconds for the tuning loop).


Mejor configuración encontrada: {'lr': 0.0001954017201375677, 'epochs': 30, 'num_layers': 1, 'hidden_size': 40}


In [ ]:
# 1) Best HP que obtuviste
best_config = {
    "lr":          0.0001954017201375677,
    "epochs":     30,
    "num_layers":  1,
    "hidden_size": 40,
}

# 2) Define y entrena el estimador con esos HP
estimator = DeepAREstimator(
    freq=FREQ,
    prediction_length=PREDICTION_LENGTH,

    # Arquitectura y LR
    num_layers=best_config["num_layers"],
    hidden_size=best_config["hidden_size"],
    lr=best_config["lr"],

    distr_output=StudentTOutput(),
    scaling=True,

    # Pasa max_epochs a través de trainer_kwargs
    trainer_kwargs={"max_epochs": best_config["epochs"]},
)

# Entrena usando TODO tu train_ds
predictor = estimator.train(train_ds)
raw_dataset = dataset  
# 3) Preparo test_ds recortando las últimas PREDICTION_LENGTH observaciones
test_ds = ListDataset(
    [
        {
            "start": item["start"],
            "target": item["target"][:-PREDICTION_LENGTH],
            "item_id": item["item_id"],
        }
        for item in raw_dataset
    ],
    freq=FREQ,
)


# 3) Genera predicciones sobre tu test real
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds,      # tu ListDataset de test “completo”
    predictor=predictor,
    num_samples=100,
)
forecasts = list(forecast_it)
tss       = list(ts_it)

# 4) Evalúa con MAE, RMSE, CRPS, lo que quieras
maes = []
for ts, fcst in zip(tss, forecasts):
    y_true = ts[-PREDICTION_LENGTH:]
    y_pred = np.mean(fcst.samples, axis=0)
    maes.append(mean_absolute_error(y_true, y_pred))
print("MAE final:", np.mean(maes))

# Si quieres métricas más completas:
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(ts_it, forecast_it)
print(agg_metrics)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\m_sam\anaconda3\envs\ts-venv\lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:71: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name  | Type        | Params | In sizes | Out sizes  
---------------------------------------------------------------
0 | model | DeepARModel | 10.2 K | ?        | [1, 100, 2]
---------------------------------------------------------------
10.2 K    Trainable params
0         Non-trainable params
10.2 K    Total params
0.041     Total estimated model params size (MB)


Epoch 0: : 50it [00:01, 46.53it/s, v_num=10, train_loss=4.100]

Epoch 0, global step 50: 'train_loss' reached 4.10063 (best 4.10063), saving model to 'c:\\Users\\m_sam\\OneDrive\\Documentos\\Maestria\\LaboIII\\TP_Final\\lab3\\notebooks\\lightning_logs\\version_10\\checkpoints\\epoch=0-step=50.ckpt' as top 1


Epoch 1: : 50it [00:01, 46.07it/s, v_num=10, train_loss=3.700]

Epoch 1, global step 100: 'train_loss' reached 3.70361 (best 3.70361), saving model to 'c:\\Users\\m_sam\\OneDrive\\Documentos\\Maestria\\LaboIII\\TP_Final\\lab3\\notebooks\\lightning_logs\\version_10\\checkpoints\\epoch=1-step=100.ckpt' as top 1


...

Epoch 29: : 50it [00:01, 44.67it/s, v_num=10, train_loss=2.630]


c:\Users\m_sam\anaconda3\envs\ts-venv\lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


MAE final: 11.578603


Running evaluation: 0it [00:00, ?it/s]


AssertionError: Some of the requested item metrics are missing.

In [ ]:
results = []
for pid, fcst in zip(pids, forecasts):
    # fcst.mean es un array de longitud PREDICTION_LENGTH
    # En tu caso prediction_length=2 => [ene2020_pred, feb2020_pred]
    pred_feb = float(fcst.mean[-1])
    results.append({
        "product_id": pid,
        "pred_tn_2020_02": pred_feb
    })

results_df = pd.DataFrame(results)
print(results_df.head())

# Escríbelo donde quieras
results_df.to_csv("../data/pronostico_feb2020_deepar_agg_V3.csv", index=False)

   product_id  pred_tn_2020_02
0       20001      1319.527954
1       20002       843.283020
2       20003       716.112183
3       20004       574.082520
4       20005       571.325256
